<a href="https://colab.research.google.com/github/cvs2010/ml_olympiad_for_students_topvistos_eua/blob/main/ML_Olympiad_for_Students_TopVistos_EUA_XGBClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# Importar o DataSet do google drive
df_sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Competitions/ML Olympiad for Students - TopVistos EUA/sample_submission.csv')
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Competitions/ML Olympiad for Students - TopVistos EUA/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Competitions/ML Olympiad for Students - TopVistos EUA/test.csv')
train['status_do_caso'] = train['status_do_caso'].map({'Negado': 0, 'Aprovado': 1})

In [7]:
train['status_do_caso'].mean()

0.6692644090603275

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, f1_score
!pip install category_encoders
from category_encoders import OrdinalEncoder
!pip install optuna
import optuna as opt

from sklearn.model_selection import StratifiedKFold

Xtrain = train.drop(['status_do_caso', 'id_do_caso'], axis=1)
ytrain = train['status_do_caso']

# Define the columns to encode
columns_to_encode = ['continente', 'educacao_do_empregado', 'tem_experiencia_de_trabalho', 
                     'requer_treinamento_de_trabalho',
                    'regiao_de_emprego', 'unidade_de_salario', 'posicao_em_tempo_integral']


def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    subsample = trial.suggest_float('subsample', 0.5, 1, step=0.1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1, step=0.1)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    
    avg_roc = 0
    avg_f1 = 0
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for tr, ts in kf.split(Xtrain, ytrain):
        Xtr, Xvl = Xtrain.iloc[tr], Xtrain.iloc[ts]
        ytr, yvl = ytrain.iloc[tr], ytrain.iloc[ts]

        encoder = OrdinalEncoder(cols=columns_to_encode)

        Xtr = encoder.fit_transform(Xtr)
        Xvl = encoder.transform(Xvl)
        
        scale_pos_weight = ytr.value_counts()[0] / ytr.value_counts()[1]
        mdl = XGBClassifier(n_estimators=1000, learning_rate=learning_rate, 
                            max_depth=max_depth, subsample=subsample, colsample_bytree=colsample_bytree, 
                            min_child_weight=min_child_weight, scale_pos_weight=scale_pos_weight, 
                            random_state=0, n_jobs=-1)
        mdl.fit(Xtr, ytr)
        p = mdl.predict_proba(Xvl)[:, 1]
        avg_roc += roc_auc_score(yvl, p)
        
        f1_66_th = np.percentile(p, 100 - 67)
        avg_f1 += f1_score(yvl, p > f1_66_th)
    

    print('avg roc:', avg_roc / 5)
    print('avg f1:', avg_f1 / 5)
    return avg_roc / 5

study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[I 2023-06-02 15:14:11,827] A new study created in memory with name: no-name-59c398e0-254f-4f26-a578-4494388450b6
[I 2023-06-02 15:14:50,044] Trial 0 finished with value: 0.7759933371882375 and parameters: {'learning_rate': 0.034125110218813634, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 3}. Best is trial 0 with value: 0.7759933371882375.


avg roc: 0.7759933371882375
avg f1: 0.8037011973008308


[I 2023-06-02 15:15:20,876] Trial 1 finished with value: 0.7557587418525774 and parameters: {'learning_rate': 0.0011031892307397436, 'max_depth': 2, 'subsample': 0.6, 'colsample_bytree': 0.9, 'min_child_weight': 3}. Best is trial 0 with value: 0.7759933371882375.


avg roc: 0.7557587418525774
avg f1: 0.7919240186594954


[I 2023-06-02 15:16:32,091] Trial 2 finished with value: 0.7809107263368453 and parameters: {'learning_rate': 0.0012872627699893231, 'max_depth': 6, 'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 1}. Best is trial 2 with value: 0.7809107263368453.


avg roc: 0.7809107263368453
avg f1: 0.8061293593824113


In [14]:
encoder = OrdinalEncoder(cols=columns_to_encode)

Xtr = train.drop(['status_do_caso', 'id_do_caso'], axis=1)
Xvl = test.drop(['id_do_caso'], axis=1)
ytr = train['status_do_caso']

Xtr = encoder.fit_transform(Xtr)
Xvl = encoder.transform(Xvl)

learning_rate = study.best_params['learning_rate']
max_depth = study.best_params['max_depth']
subsample = study.best_params['subsample']
colsample_bytree = study.best_params['colsample_bytree']
min_child_weight = study.best_params['min_child_weight']

scale_pos_weight = ytr.value_counts()[0] / ytr.value_counts()[1]

p = np.zeros((Xvl.shape[0], 5))
for seed in range(5):
    mdl = XGBClassifier(n_estimators=1000, learning_rate=learning_rate, max_depth=max_depth, 
                        subsample=subsample, colsample_bytree=colsample_bytree, 
                        min_child_weight=min_child_weight, scale_pos_weight=scale_pos_weight, 
                        random_state=seed, n_jobs=-1)
    mdl.fit(Xtr, ytr)
    p[:, seed] = mdl.predict_proba(Xvl)[:, 1]

In [18]:
# Criar o conjunto de features (X_test5) para as previsões
X_test5 = test.drop(['id_do_caso'], axis=1)

# Fazer as previsões nos dados de teste
y_pred5 = mdl.predict(Xvl)

# Criar o DataFrame com as previsões no formato do df_sample_submission
df_submission5 = pd.DataFrame(columns=['id_do_caso', 'status_do_caso'])
df_submission5['id_do_caso'] = test['id_do_caso']
df_submission5['status_do_caso'] = y_pred5

# Exibir o DataFrame com as previsões
print(df_submission5)

# Salvando o DataFrame com as previsões df_submission5 em um arquivo sample_submission.csv
df_submission5.to_csv('sample_submission.csv', index=False)

     id_do_caso  status_do_caso
0     EZYV22339               1
1      EZYV9333               0
2      EZYV9740               1
3      EZYV3328               1
4     EZYV12321               1
...         ...             ...
7639  EZYV19907               0
7640    EZYV576               0
7641   EZYV1804               1
7642  EZYV13543               1
7643  EZYV17903               1

[7644 rows x 2 columns]


In [16]:
X_test5

,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral
0,Ásia,Ensino Superior,S,N,2414,1973,Nordeste,103320.800,Ano,N
1,Ásia,Mestrado,N,S,2928,1937,Oeste,49786.350,Ano,S
2,Ásia,Ensino Superior,S,N,1836,1999,Oeste,60855.830,Ano,S
3,Ásia,Ensino Superior,S,N,122,2004,Sul,82379.060,Semana,S
4,América do Sul,Ensino Superior,S,S,1720,2003,Nordeste,6719.810,Ano,S
...,...,...,...,...,...,...,...,...,...,...
7639,América do Norte,Ensino Superior,N,N,66,2008,Oeste,70.599,Hora,S
7640,Ásia,Ensino Médio,S,N,4040,1994,Meio-Oeste,60158.590,Ano,S
7641,Ásia,Mestrado,S,N,32503,1925,Sul,44555.120,Ano,S
7642,América do Norte,Mestrado,S,N,1894,2000,Sul,97830.730,Ano,S
